In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn import metrics
import sys
import copy
import statsmodels.api as sm
pd.options.mode.chained_assignment = None
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000
%matplotlib inline

dfTemp=pd.read_csv(r'C:\Users\Jaesung Park\Desktop\FP\GroupProject\kc_house_data.csv',keep_default_na=False)

In [2]:
dfTemp = dfTemp.reindex(np.random.permutation(dfTemp.index))
#dfTemp=dfTemp.head(4000)
data= dfTemp

In [3]:
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('id')
data.price = data.price.astype(int)
data.bathrooms = data.bathrooms.astype(int)
data.floors = data.floors.astype(int)
data["house_age"] = data["date"].dt.year - data['yr_built']
data['renovated'] = data['yr_renovated'].apply(lambda yr: 0 if yr == 0 else 1)

data=data.drop('date', axis=1)
data=data.drop('yr_renovated', axis=1)
data=data.drop('yr_built', axis=1)

dataset = data[data.columns.difference(['id', 'long'])]
#X = dataset[['house_age','sqft_living','grade','condition', 
#'sqft_living15','bathrooms','view','sqft_basement','floors','waterfront',
#'renovated','bedrooms']]


In [4]:
dataset = dataset[~dataset.index.duplicated()] 

In [5]:
fall = [] # 1빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] in ([]):
        fall.append(dataset.columns[i])

five = [] # 1빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in ([ 'sqft_living']):
        five.append(dataset.columns[i])
        
fifthteen = [] # 2빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in ([ 'sqft_living','grade']):
        fifthteen.append(dataset.columns[i])

thirty = [] # 4빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['sqft_living','sqft_above','grade','sqft_living15']):
        thirty.append(dataset.columns[i])    

half = [] # 반타작
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['sqft_living','sqft_above','grade','sqft_living15',
       'bathrooms', 'view']):
        half.append(dataset.columns[i])
        
seventy = [] # 8 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['price','condition', 'house_age','zipcode', 'sqft_lot15']):
        seventy.append(dataset.columns[i]) 
        
eightyfive = [] # 10 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['price','condition', 'house_age']):
        eightyfive.append(dataset.columns[i]) 
        
ninetyfive = [] # 11 in
for i in range(len(dataset.columns)):
    if dataset.columns[i] in (['price','condition']):
        ninetyfive.append(dataset.columns[i]) 


In [6]:
testing = dataset
# linear Regression
regressor = LinearRegression(fit_intercept=True)
#alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1, 10, 15, 20, 30, 40, 100]
#regressor=RidgeCV(alphas=alphas, store_cv_values=True)

In [7]:
testing.shape[0]

3996

In [8]:
## exclude the lefts
rounded = round(testing.shape[0]/10)*10
left = testing.shape[0] - rounded
if left < 0:
    left = abs(left)
testing2 = testing.iloc[left:]

k = testing2.shape[0]
fold = 10
point = round(k/fold)
mylist = []

start = 0
end= point

# separating dataset into # number of folds and put each fold into list
for i in range(fold):
    x = testing2.iloc[start:end]
    start = start + point
    end = end + point
    mylist.append(x)

In [9]:
KNN_MAE = []
KNN_MSE = []
KNN_RMSE = []
r2_list = []
temp = copy.deepcopy(mylist)
fffinal = []

In [10]:
#aaa= testing_set.iloc[3].to_numpy()
def dist(x,y):
    return np.sqrt(np.sum((x-y)**2))


In [11]:
hm = five
flag = 0
iflag = 0
trueFalse = []
# L is K value for KNN
L = 10
for j in range(fold):
    predicted_MAE = []
    predicted_MSE = []
    predicted_RMSE = []
    predicted_r2 = []
    mylist = copy.deepcopy(temp)
    testing_set = mylist[j]
    appended_data = []
    euclidean_list = []
    for z in range(fold):
        if not z == j:
            appended_data.append(mylist[z])
    training_set = pd.concat(appended_data, ignore_index = True)
    #training and testing set is created by each fold
    if flag == 0:
        for p in range(len(testing_set.columns)):
            # putting false to unknown columns, run only one time
            if testing_set.columns[p] not in hm:
                trueFalse.append(True)
            else:
                trueFalse.append(False)
        finalTF = np.array(trueFalse)
        flag = 1
        
    nptraining_set = training_set.to_numpy() 
    nptesting_set = testing_set.to_numpy()
    knownTraining_set = nptraining_set[:,finalTF]
    knownTesting_set = nptesting_set[:,finalTF]
    
    for m in range(L): #K
        for q in range(len(knownTesting_set)):#row for the testing set
            euclidean_list = []
            for qq in range(len(knownTraining_set)): #row for the training set
                euc = dist(knownTesting_set[q], knownTraining_set[qq])
                euclidean_list.append(euc)
            
            # The below lamda method is from stackoverflow, basically it's telling the original index number while sorting
            eu_index = sorted(range(len(euclidean_list)), key=lambda k: euclidean_list[k])
            for z in range(len(testing_set.columns)): # column
                if testing_set.columns[z] in hm and testing_set.columns[z] != 'price':
                    testing_set.iat[q,z] = training_set.iat[eu_index[m],z]
                
                    
        
        testing_set_X = testing_set[testing.columns.difference(['price'])]
        testing_set_Y = np.log(testing_set['price'])
        training_set_X = training_set[testing.columns.difference(['price'])]
        training_set_Y = np.log(training_set['price'])
        
        regressor.fit(training_set_X, training_set_Y)
        y_pred = regressor.predict(testing_set_X)
        MAE = metrics.mean_absolute_error(testing_set_Y, y_pred)
        MSE = metrics.mean_squared_error(testing_set_Y, y_pred)
        RMSE = np.sqrt(metrics.mean_squared_error(testing_set_Y, y_pred))
        r2 = metrics.r2_score(testing_set_Y, y_pred)
       
        predicted_MAE.append(MAE)
        predicted_MSE.append(MSE)
        predicted_RMSE.append(RMSE)
        predicted_r2.append(r2)
       

    predicted_MAE.sort()
    predicted_MSE.sort()
    predicted_RMSE.sort()
    predicted_r2.sort(reverse=True)

    #KNN_MAE.append(predicted_MAE[0])
    #KNN_MSE.append(predicted_MSE[0])
    #KNN_RMSE.append(predicted_RMSE[0])
    #r2_list.append(predicted_r2[0])
    KNN_MAE.append(np.mean(predicted_MAE))
    KNN_MSE.append(np.mean(predicted_MSE))
    KNN_RMSE.append(np.mean(predicted_RMSE))
    r2_list.append(np.mean(predicted_r2))
        
        

                
        


    
    


In [12]:
coeff_df = pd.DataFrame(regressor.coef_, training_set_X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
bathrooms,2.947568e-02
bedrooms,5.747171e-03
condition,7.435068e-02
floors,6.962530e-02
grade,1.697059e-01
house_age,3.015542e-03
lat,1.419717e+00
renovated,1.082309e-01
sqft_above,3.594324e-05
sqft_basement,4.840200e-05


In [13]:
np.mean(r2_list)

0.16762305437394232

In [14]:
np.mean(KNN_MAE)

0.37931238070492634

In [15]:
np.mean(KNN_MSE)

0.2242064150030137

In [16]:
np.mean(KNN_RMSE)

0.4731994238394475